In [1]:
# This code allows a speech signal (i.e., source) to imitate another speech signal (i.e., target)
# using signal processing. For this example, audio1 is the source and audio2 is the target.
# Author: Dora Maria Ballesteros.
# 26/05/2021.

from scipy.io import wavfile
import IPython
import numpy as np
import scipy.stats as stats
from scipy.stats import kurtosis, skew, entropy
import matplotlib.pyplot as plt
import numpy as np
import pywt
from pywt import wavedec
from numpy import savetxt

In [2]:
# 1. Load the audios and scaling
sr1, source = wavfile.read('/content/audio1.wav')
source=source/(np.max(abs(source)))
IPython.display.Audio(source, rate=sr1)

In [3]:
sr2, target = wavfile.read('/content/audio2.wav')
target=target/(np.max(abs(target)))
IPython.display.Audio(target, rate=sr2)

In [4]:
#2. Apply DWT 
csource = wavedec(source, 'sym4', level=2)
ctarget = wavedec(target, 'sym4', level=2)

csA2 = csource[0]
csD2 = csource[1]
csD1 = csource[2]

ctA2 = ctarget[0]
ctD2 = ctarget[1]
ctD1 = ctarget[2]

cs=np.concatenate((csA2, csD2, csD1), axis=0)
ct=np.concatenate((ctA2, ctD2, ctD1), axis=0)

In [5]:
#3. Apply Sort
csource_sorted =np.sort(cs)
index1 = np.argsort(cs)

ctarget_sorted= np.sort(ct)
index2 = np.argsort(ct)

In [6]:
# 4. Assign new positions to the wavelet coefficients of audio1, according to index1 and index2.

# the rule is coeff_source[index_target]=coeff_source[index_source]
cs_m=np.zeros(len(cs))
cs_m[index2]=cs[index1]

l1 = len(csource[0])
l2 = len(csource[1])
l3 = len(csource[2])

csource_m=csource
csource_m[0] = cs_m[0:l1]
csource_m[1] = cs_m[l1:l1+l2]
csource_m[2] = cs_m[l1+l2:l1+l2+l3]

# And obtain and save the key according to index1 and index2.

key =np.zeros(len(cs))
key[index1]=index2
savetxt('key.csv', key, delimiter=',')

In [7]:
# 5. Apply IDWT

audio1_I = pywt.waverec(csource_m, 'sym4')

# listen the "new" audio1 (voice conversion from audio1 to audio2) 
IPython.display.Audio(audio1_I, rate=sr1)